## Variations of dict (Một vài biến thể của dict)

An overview of mapping types included in the standard library, besides `defaultdict`

### 1. collections.OrderedDict

The most common reason to use OrderedDict is writing code that is backward compatible (tương thích ngược) with earlier Python versions

### 2. collections.ChainMap

A ChainMap instance: holds a list of mappings that can be searched as one.

In [4]:
# The search is performed on each input mapping in the order and succeeds as soon as the key is found

d1 = dict(a=1, b=3)
d2 = dict(a=2, b=4, c=6)
from collections import ChainMap
chain = ChainMap(d1, d2)
print(chain['a'])

# The ChainMap instance does not copy the input mappings, but holds references to them.
# Updates or insertions to a ChainMap only affect the first input mapping.
chain['c'] = -1
print(d1)
print(d2)

1
{'a': 1, 'b': 3, 'c': -1}
{'a': 2, 'b': 4, 'c': 6}


ChainMap rất hữu ích để triển khai trình thông dịch cho các ngôn ngữ có phạm vi lồng nhau, trong đó mỗi mapping biểu thị một scope contex, từ phạm vi bao quanh trong cùng đến phạm vi ngoài cùng.

In [5]:
import builtins
pylookup = ChainMap(locals(), globals(), vars(builtins))

### 3. collections.Counter

A mapping that holds an integer count for each key. Updating an existing key adds to its count.
This can be used to *count instances of hashable objects or as a multiset* (discussed later in this section).

In [6]:
from collections import Counter
ct = Counter('abracadabra')
print(ct)

ct.update('aaaaazzz')
print(ct)
ct.most_common(3)

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})
Counter({'a': 10, 'z': 3, 'b': 2, 'r': 2, 'c': 1, 'd': 1})


[('a', 10), ('z', 3), ('b', 2)]

### 4. shelve.Shelf
Provides persistent (lien tuc) storage for a mapping of string keys to Python objects serialized in the `pickle` binary format

*WARNING*
Python’s `pickle` is easy to use in the simplest cases, but has several drawbacks. Read Ned Batchelder’s “Pickle’s nine flaws” (https://nedbatchelder.com/blog/202006/pickles_nine_flaws.html) before adopting any solution involving `pickle`. In his post, Ned mentions other serialization formats to consider.

`OrderedDict`, `ChainMap`, `Counter`, and `Shelf` are ready to use but can also be customized by subclassing. In contrast, `UserDict` is intended only as a base class to be extended.

### 5. Subclassing UserDict Instead of dict
Lý do chính mà dùng `UserDict` thay vì `dict` là vì tích hợp sẵn có một số implementation shortcuts buộc chúng ta phải ghi đè các phương thức mà chúng ta có thể kế thừa từ `UserDict` mà không gặp vấn đề gì.

Lưu ý rằng `UserDic`t không kế thừa từ `dict`, nhưng sử dụng thành phần: nó có một `internal dict instance`, called `data`, chứa các actual items

In [7]:
# StrKeyDict always converts nonstring keys to str on insertion, update, and lookup
import collections


class StrKeyDict(collections.UserDict): # StrKeyDict extends UserDict

    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]

    def __contains__(self, key):         # __contains__ is simpler: we can assume all stored keys are str, and we can check on self.data instead of invoking self.keys() as we did in StrKeyDict0
        return str(key) in self.data

    def __setitem__(self, key, item):    # __setitem__ converts any key to a str. This method is easier to overwrite when we can delegate to the self.data attribute.
        self.data[str(key)] = item

Because `UserDict` extends `abc.MutableMapping`, the remaining methods that make `StrKeyDict` a full-fledged (đầy đủ) mapping are inherited from `UserDict`, `MutableMapping`, or `Mapping`
--> Have several useful concrete (cụ thể) methods
  - `MutableMapping.update`
  - `Mapping.get`

We know there are immutable sequence types, but how about an immutable mapping? Well, there isn’t a real one in the standard library, but a stand-in is available.

## Immutable Mappings

The mapping types provided by the standard library are all mutable, but you may need to prevent users from changing a mapping

--> The `types` module provides a wrapper(bọc) class called `MappingProxyType`, which, given a mapping, returns a `mappingproxy` instance that is a _read-only_ but dynamic proxy for the original mapping. This mean, `updates` to the original mapping can be seen in the mappingproxy, but changes cannot be made through it.

Ví dụ thực tế: Hàm tạo trong một subclass `Board` sẽ fill một private mapping với pin object, và hiển thị nó cho API clientsthông qua thuộc tính `.pins` công khai được triển khai dưới dạng `mappingproxy` --> clients ko thể thêm hay xoá.



In [8]:
# Example 3-10. MappingProxyType builds a read-only mappingproxy instance from a dict

from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)

d_proxy[2] = 'x' # Update ?

TypeError: 'mappingproxy' object does not support item assignment

In [9]:
d[2] = 'B' # Update
d_proxy

mappingproxy({1: 'A', 2: 'B'})

## Dictionary Views

allow high-performance operations on a dict, without unnecessary copying of data.

The dict instance methods `.keys()`, `.values()`, and `.items()` return instances of classes called `dict_keys`, `dict_values`, and `dict_items`

These _dictionary views_ are read-only projections (phép chiếu) of the internal data structures used in the dict implementation --> avoid the memory overhead of the equivalent Python 2 methods that returned lists duplicating data

In [13]:
# Example 3-11. The .values() method returns a view of the values in a dict

d = dict(a=10, b=20, c=30)
values = d.values()
values

dict_values([10, 20, 30])

In [14]:
print(len(values))

3


In [15]:
# Views are iterable, so it’s easy to create lists from them
list(values)

[10, 20, 30]

In [16]:
# Views implement __reversed__, returning a custom iterator.
reversed(values)

In [17]:
# We can’t use [] to get individual items from a view.
values[0]

TypeError: 'dict_values' object is not subscriptable

In [18]:
# A view object is a dynamic proxy. If the source dict is updated, you can immediately see the changes through an existing view.
d['z'] = 99
values

dict_values([10, 20, 30, 99])

## Practical Consequences of How dict Works

The hash table implementation of Python’s `dict` is very efficient, but it’s important to understand the practical effects of this design:
- Keys must be hashable objects.
- Item access by key is very fast, by computing the hash code of the key and deriving an index offset
- Key ordering is preserved (được bảo vệ)
- Although its new compact layout, dicts inevitably have a significant memory overhead.
- To save memory, avoid creating instance attributes outside of the `__init__` method.

## Set Theory

NOTE
In this book, I use the word “set” to refer both to `set` and `frozenset`. When talking specifically about the `set` class, I use constant width font: `set`.

A set is a collection of unique objects.

In [19]:
l = ['spam', 'spam', 'eggs', 'spam', 'bacon', 'eggs']
set(l)

{'bacon', 'eggs', 'spam'}

In [20]:
# If you want to remove duplicates but also preserve the order of the first occurrence
dict.fromkeys(l).keys()
list(dict.fromkeys(l).keys())

['spam', 'eggs', 'bacon']

Set elements must be hashable.

The `set` type is not hashable, so you can’t build a `set` with nested `set` instances.

But `frozenset` is hashable, so you can have `frozenset` elements inside a `set`.

Sử dụng Set thông minh có thể giảm lượng code và chaỵ nhanh hơn.
a | b returns their union,
a & b computes the intersection,
a - b the difference,
a ^ b the symmetric difference

VD: Have a large set of email addresses (the haystack) and a smaller set of addresses (the needles)
Need to count how many needles occur in the haystack ? --> Use `&`

`found = len(needles & haystack)` --> need set object


```
found = len(set(needles) & set(haystack))

# another way:
found = len(set(needles).intersection(haystack))
```

=

```
found = 0
for n in needles:
    if n in haystack:
        found += 1
```

### 1. Set Literals
The syntax of `set` literals—`{1}`, `{1, 2}`
To create an empty `set`, you should use the constructor without an argument: `set()`. If you write `{}`, you’re creating an empty `dict`

In [21]:
s = {1}
type(s)

set

In [22]:
s = {}
type(s)

dict

Literal set syntax like `{1, 2, 3}` is both faster and more readable than calling the constructor (e.g., `set([1, 2, 3])`)

In [23]:
frozenset(range(10)) # frozenset must be created by calling the constructor.

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

Speaking of syntax, the idea of listcomps was adapted to build sets as well.

### 1.2. Set Comprehensions (bao quat)
Set comprehensions (setcomps)

In [25]:
from unicodedata import name
name(chr(36),'')

'DOLLAR SIGN'

In [26]:
chr(36)

'$'

In [29]:
# Example 3-15. Build a set of Latin-1 characters that have the word “SIGN” in their Unicode names
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i),'')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

The order of the output changes for each Python process, because of the salted hash mentioned in “What Is Hashable”.

## Practical Consequences of How Sets Work

The `set` and `frozenset` types are both implemented with a hash table. This has these effects:
- Set elements must be hashable objects. They must implement proper `__hash__` and `__eq__` methods as described in “What Is Hashable”.
- Membership testing is very efficient: an element can be located directly by computing its hash code and deriving an index offset
- Sets have a significant memory overhead
- Element ordering depends on insertion order, but not in a useful or reliable way.
- Adding elements to a set may change the order of existing elements. (when 2/3 fulls, resize the table)

### 1. Set Operations

Figure 3-2 gives an overview of the methods you can use on mutable and immutable sets. Note that some operators and methods perform in-place changes on the target set (e.g., &=, difference_update, etc.)

![img.png](image/3-2.png)


More about set's API: https://learning.oreilly.com/library/view/fluent-python-2nd/9781492056348/ch03.html#set_uml

## Set Operations on dict Views

Table 3-5 (https://learning.oreilly.com/library/view/fluent-python-2nd/9781492056348/ch03.html#view_methods_tbl) shows that the view objects returned by the `dict` methods .keys() and .items() are remarkably similar to `frozenset`.

In particular, `dict_keys` and `dict_items` implement the special methods to support the powerful set operators & (intersection), | (union), - (difference), and ^ (symmetric difference).

In [30]:
# using & is easy to get the keys that appear in two dictionaries
d1 = dict(a=1, b=2, c=3, d=4)
d2 = dict(b=20, d=40, e=50)
d1.keys() & d2.keys() # return set

{'b', 'd'}

In [31]:
# the set operators in dictionary views are compatible (tuong thich) with set instances.
s = {'a', 'e', 'i'}
print(d1.keys() & s)
print(d1.keys() | s)

{'a'}
{'e', 'i', 'c', 'd', 'b', 'a'}


WARNING
A dict_items view only works as a set if all values in the dict are hashable.

# Chapter Summary

Dictionaries are a keystone of Python. Over the years, the familiar `{k1: v1, k2: v2}` literal syntax was enhanced to support unpacking with `**`, pattern matching, as well as `dict` comprehensions

Beyond the basic `dict`, the standard library offers handy, ready-to-use specialized mappings like `defaultdict`, `ChainMap`, and `Counter`, all defined in the `collections` module. With the new dict implementation, `OrderedDict` is not as useful as before. Also in the collections module is the `UserDict`, an easy to use base class to create custom mappings

Two powerful methods available in most mappings are `setdefault` and `update`
- The `setdefault` method can update items holding mutable values—for example, in a `dict` of `list` values, avoiding a second search for the same key
- The `update` method allows bulk insertion or overwriting of items. Since Python 3.9, we can also use the `|=` operator to update a mapping, and the `|` operator to create a new one from the union of two mappings

The `__missing__` method, which lets you customize what happens when a key is not found when using the `d[k]` syntax that invokes `__getitem__`.

The `collections.abc` module provides the `Mapping` and `MutableMapping` abstract base classes as standard interfaces, useful for runtime type checking. The `MappingProxyType` from the types module creates an immutable map, avoid users update. There are also ABCs for `Set` and `MutableSet`

Dictionary views were a great addition in Python 3, eliminating the memory overhead of the Python 2 `.keys()`, `.values()`, and `.items()` methods. In addition, the `dict_keys` and `dict_items` classes support the most useful operators and methods of `frozenset`